# Homework 7: An MITgcm Tutorial Experiment

Name: 

Date: 

#### Overview
In class, we walked through the MITgcm tutorial for the barotropic gyre. In this homework, you'll work through another experiment - the `tutorial_global_oce_latlon` configuration. 

Documentation for this tutorial is additionally provided on the MITgcm documentation page [HERE](https://mitgcm.readthedocs.io/en/latest/examples/global_oce_latlon/global_oce_latlon.html).

Before beginning, import Python modules for manging and plotting data, and define a path to the model directory on your machine:

In [1]:
# import pertinent modules
import os
import numpy as np
import matplotlib.pyplot as plt

# define a path to the model directory
model_dir = '/Users/harsita/Desktop/sjsu/year 3/spring 25/cs 185c - ocean modeling/MITgcm/verification/tutorial_global_oce_latlon'

### Part 1: Plot the model bathymetry
One good way to examine the construction of a model is to plot the bathymetry. The bathymetry for this model is stored in the `input/bathymetry.bin` file. Create a plot of the bathymetry below. Be sure to label your axes and give the plot a colorbar with a sensible color map.

Hint: The grid is stored as a 2D grid with `float32` precision. Look at the `SIZE.h` file to determine the grid shape. 

In [3]:
# read in the model bathymetry file and reshape to the grid dimensions
n_rows = 40
n_cols = 90
dx = 20000
dy = 20000
x = np.arange(0,n_cols*dx,dx)
y = np.arange(0,n_rows*dy,dy)
X,Y = np.meshgrid(x,y)
bathy_file = os.path.join(model_dir, 'input', 'bathymetry.bin')
bathy_grid = np.fromfile(bathy_file, '>f4').reshape((n_rows,n_cols))

# make a plot of the bathymetry
fig = plt.figure(figsize = (5,4))
C = plt.pcolormesh(bathy_grid)
plt.colorbar(C)
plt.xlabel('columns')
plt.ylabel('rows')
plt.title('model bathymetry')
plt.show()

### Part 2: Compile the Model
Next, compile the model on your machine. 

Before you begin your compilation, make two changes to the files in the code directory:
1. In the packages.conf file, remove the lines for `mnc`, `timeave`, and `ptracers` and add a new line for `diagnostics`.
2. Add the `DIAGNOSTICS_SIZE.h` file provided with this notebook.

Once you make these changes, compile the model code.

Refer to the class example or the [Getting Started with MITgcm](https://profmikewood.github.io/ocean_modeling_book/mitgcm/getting_started_with_MITgcm.html) section of the course notes for help.

#### Question:
How many CPUs will this model use on your machine?

Answer: 

nPx * nPy = 1 * 1 = 1 CPU

### Part 3: Run the Model
After your compilation is complete, run the model in the `run` directory. 

Before running this model, make the following updates to the default tutorial:
1. In the `data` file, change `nIter=0` to `nIter=70080` and place the the `pickup.0000070080.data`, `pickup.0000070080.meta`, `pickup_cd.0000070080.data` and `pickup_cd.0000070080.meta` files provided with this notebook in your run directory. This will allow you to start the model after it has already been run for several decades.
2. Update the model timesteps to run for a duration of one year. Implement this change by looking in the `data` file to determine the number of seconds for each model timesteps (`deltaTClock`). Then, compute how many timesteps will be needed to run the model for one year by determining how many seconds are in one year (365 days) and dividing by the timestep. Then, update the timesteps on the line `nTimesteps=...,` in the `data` file.
3. In the `data` file, add a new line for `useSingleCpuIO=.TRUE.,` in the `PARM01` section just below the line that says `readBinaryPrec=32,`.
4. In the `data` file, in `PARM03`, update the following three lines as follows:
```
 pChkptFreq= 31536000.,
# dumpFreq=   864000.,
# taveFreq=   864000.,
```
   Note that the comment indicators (`#`) must be in the first column.
   
5. Add the `data.diagnostics` file provided with this notebook into your run directory. This will request the model to output monthly averages of temperature (`THETA`), salinity (`SALT`), zonal velocity (`UVEL`), and meridional velocity (`VVEL`).
6. In the file `data.pkg`, remove the line for `usePTRACERS=.TRUE.,` and add a new line for `useDIAGNOSTICS=.TRUE.,`.

After making these changes, run the model. 

Again, you can refer to the class example or the [Getting Started with MITgcm](https://profmikewood.github.io/ocean_modeling_book/mitgcm/getting_started_with_MITgcm.html) section of the course notes for help.

#### Questions:

1. What is the model timestep in seconds? days?
2. How many timesteps are required to run the model for one year?

Answers: *Enter your answers here*
1. One timestep is 864000 seconds which equals to one day.
2. To run the model for one year, 365 timesteps are required.

### Part 4: Assess the Model Results
Upon successful completion of your model run, you should have the following files: `THETA.0000070445.data`, `SALT.0000070445.data`, `UVEL.0000070445.data`, and `VVEL.0000070445.data`. These files contain the surface fields for temperature, salinity, zonal velocity, and meridional velocity in the month of December. 

Read these files into four separate grids and make a 2x2 set of subplots showing the surface temperature, salinity, zonal velocity, and meridional velocity simulated by the model. 

In [4]:
# read in the data from files into grids for theta, salt, uvel, and vvel
theta_file = os.path.join(model_dir, 'run', 'THETA.0000070445.data')
salt_file = os.path.join(model_dir, 'run', 'SALT.0000070445.data')
uvel_file = os.path.join(model_dir, 'run', 'UVEL.0000070445.data')
vvel_file = os.path.join(model_dir, 'run', 'VVEL.0000070445.data')

t_grid = np.fromfile(theta_file, '>f4').reshape(n_rows,n_cols)
s_grid = np.fromfile(salt_file, '>f4').reshape(n_rows,n_cols)
u_grid = np.fromfile(uvel_file, '>f4').reshape(n_rows,n_cols)
v_grid = np.fromfile(vvel_file, '>f4').reshape(n_rows,n_cols)

<>:7: SyntaxWarning: invalid escape sequence '\l'
<>:15: SyntaxWarning: invalid escape sequence '\l'
<>:23: SyntaxWarning: invalid escape sequence '\l'
<>:31: SyntaxWarning: invalid escape sequence '\l'
<>:7: SyntaxWarning: invalid escape sequence '\l'
<>:15: SyntaxWarning: invalid escape sequence '\l'
<>:23: SyntaxWarning: invalid escape sequence '\l'
<>:31: SyntaxWarning: invalid escape sequence '\l'
/tmp/ipykernel_1027/3681203037.py:7: SyntaxWarning: invalid escape sequence '\l'
  label='$\longleftarrow$ degrees celcius $\longrightarrow$' )
/tmp/ipykernel_1027/3681203037.py:15: SyntaxWarning: invalid escape sequence '\l'
  label='$\longleftarrow$ salinity units $\longrightarrow$' )
/tmp/ipykernel_1027/3681203037.py:23: SyntaxWarning: invalid escape sequence '\l'
  label='$\longleftarrow$ westward   (m/s)   eastward $\longrightarrow$' )
/tmp/ipykernel_1027/3681203037.py:31: SyntaxWarning: invalid escape sequence '\l'
  label='$\longleftarrow$ southward   (m/s)   northward $\longright

Text(0.5, 0, 'columns')

In [7]:
# make the 2x2 plot here
fig = plt.figure(figsize=(13, 11))

plt.subplot(2,2,1)
C=plt.pcolormesh(t_grid, vmin=-2, vmax=32)
plt.colorbar(C, orientation='vertical',
             label='$\longleftarrow$ degrees celcius $\longrightarrow$' )
plt.title('theta')
plt.ylabel('rows')
plt.xlabel('columns')

plt.subplot(2,2,2)
C=plt.pcolormesh(s_grid, vmin=32, vmax=35)
plt.colorbar(C, orientation='vertical',
             label='$\longleftarrow$ salinity units $\longrightarrow$' )
plt.title('salt')
plt.ylabel('rows')
plt.xlabel('columns')

plt.subplot(2,2,3)
C=plt.pcolormesh(u_grid, vmin=-.1, vmax=.1, cmap='seismic')
plt.colorbar(C, orientation='horizontal',
             label='$\longleftarrow$ westward   (m/s)   eastward $\longrightarrow$' )
plt.title('UVEL')
plt.ylabel('rows')
plt.xlabel('columns')

plt.subplot(2,2,4)
C=plt.pcolormesh(v_grid, vmin=-.1, vmax=.1, cmap='seismic')
plt.colorbar(C, orientation='horizontal',
             label='$\longleftarrow$ southward   (m/s)   northward $\longrightarrow$' )
plt.title('VVEL')
plt.ylabel('rows')
plt.xlabel('columns')

#### Questions

How do these fields compare to those from Lecture 2-1 and Homework 2? What features are consistent between the models? What features are different? Identify at least two real oceanographic features simulated in this simple model of the ocean.

Answer: 
For UVEL and VVEL, there is a noticeable difference in the level of detail. In this case, a lower minimum and maximum value (vmin and vmax) are needed for an accurate comparison. However, the overall direction of flow remains consistent with the previous notebooks.  

In the theta plot, the general pattern is also similar, but there is a cutoff near the North Pole, affecting the representation of longitude in that region.  

Two real oceanographic features simulated in this simple ocean model are:  
1) Ocean Gyres – These illustrate how currents circulate and interact with temperature variations.  
2) Salinity Distribution – This highlights variations in salt concentration, influencing ocean circulation and density.